<p style="text-align:right"><img  src="https://postmba.org/www/wp-content/uploads/2021/10/PostMBA-logo.png" width="150" alt="regression"></p>


<p style="text-align:center"><img  src="utils/images/sarima_model.jpg" width="600" alt="regression"></p>


<p dir=rtl style="direction: rtl;text-align: center;line-height:200%;font-family:vazir;font-size:medium;align:left">
<font face="Arial" size=3><b>
Created by :
Hosein Ahmadi</b>
</font>
</p>


### 1- import packages

In [154]:
import numpy as np
import pmdarima
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import datetime as datetime
import pandas_datareader as pdr
import time
from scipy import signal
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

### 2- define functions and classes

In [ ]:
def get_opt_diff(pd_series):
    '''
        function fot best diff for get confidence about stationary of the series with adfuler test
    '''
    
    adf_pvalue = adfuller(pd_series)[1]
    opt_diff = 0
    while True:
        if adf_pvalue > 0.05:
            opt_diff += 1
            pd_series = (pd_series - pd_series.shift(1)).dropna()
            adf_pvalue = adfuller(pd_series)[1]
        else:
            break
    return opt_diff

def get_model_mesage(model_num, p, opt_diff, q, P, D, Q, seasonality, model_aic, duration):
    '''
        a function for print message of the searchong on the models
    '''
    
    msg = (f'Model No. {model_num} ARIMA ({p}, {opt_diff}, {q})({P}, {D}, {Q}) [{seasonality}], AIC = {model_aic}, time = {duration} sec')
    print (msg)

def calculate_best_sarima_model(series, pmax, qmax, Pmax, Qmax, seasonality):
    '''
        a function for searching on models of SARIMA model
    '''
    
    model_count = 0
    best_aic = np.inf
    opt_diff = get_opt_diff(pd_series = series)
    
    seasonality_test = pmdarima.arima.OCSBTest(seasonality, lag_method='aic', max_lag=36)
    D = seasonality_test.estimate_seasonal_differencing_term(series)
    
    for p in range(0, pmax+1):
        for q in range(0, qmax+1):
            for P in range(0, Pmax+1):
                for Q in range(0, Qmax+1):
                    t1 = time.time()
                    model_count += 1
                    arima_model = ARIMA(series,
                                        exog = None,
                                        order = (p, opt_diff, q),
                                        seasonal_order=(P, 1, Q, seasonality),
                                        enforce_stationarity=False,
                                        enforce_invertibility=False,
                                        trend=(0, 0, 1, 0))
                    fitted_arima = arima_model.fit()
                    model_aic = fitted_arima.aic
                    if model_aic<best_aic:
                        best_aic = model_aic
                        optimal_p = P
                        optimal_q = q
                        optimal_P = P
                        optimal_Q = Q
                        
                    t2 = time.time()
                    
                    get_model_mesage(model_count, p, opt_diff,
                                     q, P, D, Q,
                                     seasonality, round(model_aic, 1),
                                     round(t2-t1, 1))
                    
    best_model_params = {'optimal_p': optimal_p, 'optimal_q': optimal_q,
                  'optimal_P': optimal_P, 'optimal_Q': optimal_Q,
                  'opt_diff': opt_diff}
    
    #print best model
    best_model = ARIMA(series,
                        exog = None,
                        order = (optimal_p, opt_diff, optimal_q),
                        seasonal_order=(optimal_P, 1, optimal_Q, seasonality),
                        enforce_stationarity=False,
                        enforce_invertibility=False,
                        trend=(0, 0, 1, 0))
    
    fitted_model = best_model.fit()
    # print(fitted_model.summary)
    # yf = fitted_arima.forecast(fsteps, exog = None)
    return fitted_model, best_model_params

In [150]:
class sarima_model:
    '''
        main class
    '''
    
    def __init__(self, series, pmax, qmax, Pmax, Qmax, seasonality):
        self.series = series
        self.pamx = pmax
        self.qmax = qmax
        self.Pmax = Pmax
        self.Qmax = Qmax
        self.seasonality = seasonality
        
    def get_optimal_sarima_model(self):
        self.best_model, self.best_params = calculate_best_sarima_model(self.series, self.pamx, self.qmax,
                                                      self.Pmax, self.Qmax, self.seasonality)
        
        print('\n ================= best model ====================')
        print(self.best_params)
        print(self.best_model.summary())
        
        return self.best_model
        
    def forecast(self, fsteps):
        return self.best_model.forecast(fsteps, exog = None)

## 3- modelling

- GDP

In [161]:
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime(2024, 1, 1)
gdp_data = pdr.data.DataReader('GDP', data_source = 'fred', start = start, end = end)
fig = px.line(gdp_data)
fig.update_layout(template='simple_white', width=800, height=400)
fig.show()

In [151]:
pmax  = 1 #maximum lag on regular auto regressive
qmax = 1 #maximum lag on regular ma
Pmax = 1 #maximum lag on seasonalty auto regressive
Qmax = 1 #maximum lag on seasonality moving average
fsteps = 12 #inteval between model optimization
fstepsout = 12 #maxium steps on model searching
seasonality = 4

model = sarima_model(gdp_data['GDP'], pmax, qmax, Pmax, Qmax, seasonality)
best_model = model.get_optimal_sarima_model()

Model No. 1 ARIMA (0, 1, 0)(0, 0, 0) [4], AIC = 3994.4, time = 0.1 sec
Model No. 2 ARIMA (0, 1, 0)(0, 0, 1) [4], AIC = 3765.5, time = 0.1 sec
Model No. 3 ARIMA (0, 1, 0)(1, 0, 0) [4], AIC = 3878.9, time = 0.0 sec
Model No. 4 ARIMA (0, 1, 0)(1, 0, 1) [4], AIC = 3767.2, time = 0.1 sec
Model No. 5 ARIMA (0, 1, 1)(0, 0, 0) [4], AIC = 3975.6, time = 0.1 sec
Model No. 6 ARIMA (0, 1, 1)(0, 0, 1) [4], AIC = 3753.8, time = 0.2 sec
Model No. 7 ARIMA (0, 1, 1)(1, 0, 0) [4], AIC = 3876.8, time = 0.1 sec
Model No. 8 ARIMA (0, 1, 1)(1, 0, 1) [4], AIC = 3755.7, time = 0.2 sec
Model No. 9 ARIMA (1, 1, 0)(0, 0, 0) [4], AIC = 3987.4, time = 0.0 sec
Model No. 10 ARIMA (1, 1, 0)(0, 0, 1) [4], AIC = 3765.8, time = 0.2 sec
Model No. 11 ARIMA (1, 1, 0)(1, 0, 0) [4], AIC = 3863.6, time = 0.1 sec
Model No. 12 ARIMA (1, 1, 0)(1, 0, 1) [4], AIC = 3767.7, time = 0.2 sec
Model No. 13 ARIMA (1, 1, 1)(0, 0, 0) [4], AIC = 3976.4, time = 0.0 sec
Model No. 14 ARIMA (1, 1, 1)(0, 0, 1) [4], AIC = 3754.8, time = 0.2 sec
M

In [152]:
best_model.forecast(fsteps=10)

2023-10-01    27884.352213
Freq: QS-OCT, dtype: float64